In [1]:
import pygad
import numpy
import random
from pomegranate import DiscreteDistribution, HiddenMarkovModel
from hmm.hmm import ChromosomeHMM, normalize_rows, hmm_from_vector, random_chromosome
from ga.crossover import single_row_cutpoint
from ga.mutation import single_point_mutation

In [3]:
N_STATES = 6
ALPHABET = list('ABCDEFG')

parent_chromosomes = random_chromosome(N_STATES, ALPHABET)
parent_hmm = hmm_from_vector(parent_chromosomes, N_STATES, ALPHABET)

# parent_training_sequences = numpy.random.choice(ALPHABET, (10, 50))
# parent_hmm.fit(parent_training_sequences)

# Genetic Algorithm Steps
def fitness_func(solution, solution_idx):
    # Wichtig!!!!!!!!!!!!!!!!!! zweiter parameter muss gesetzt werden sonst endlos loop
    samples = parent_hmm.sample(1, 50)
    # !!!!!!!!!!!!!!!!!! Die Parameter vom Child HMM sind nicht alle in range [0,1]
    # manche negativ manche größer als 1
    # vielleicht hab ich irgendwo reingespastet und die random generation ist nicht in der range [0,1]
    child_hmm = hmm_from_vector(solution, N_STATES, ALPHABET)

    total_score = []
    for sample in samples:
        total_score.append(child_hmm.log_probability(sample))

    mean_score = sum(total_score)/len(samples)
    if mean_score > 0:
        x = child_hmm.forward_backward(sample)
        print('shish')
    return mean_score








# Wichtig!!!!!
# Die Crossover Funktion wird 1 mal pro generation aufgerufen
# Offspring_size[0] ist die Population_size - keep_elitism (wenn keep_elitism > 0 ist, sonst abhängig von keep_parents)

# Legal Cut Points = num_symbols, num_states






# Random Child Vector
# possible_solution = generate_random_solution_vector(N_STATES, ALPHABET)
# print(possible_solution)

# number of solutions to be selected as parent


num_parents_mating=2

# Initial Population (can not be generated py pygad because elements have to be row-stochastic)
# Equivalent to sol_per_pop
population_size = 5
initial_population = [random_chromosome(N_STATES, ALPHABET) for i in range(population_size)]


# Values can be only in range [0,1]
init_range_low = 0
init_range_high = 1

# Evaluate fitness of vector
# fitness = fitness_func(possible_solution, 0)
# print(fitness)

# def on_stop(ga_instance, res):
#     print(res)

# cut_points = 

crossover_func = single_row_cutpoint(N_STATES, len(ALPHABET))
def mutation_func(offspring, ga_instance):
    return offspring

ga_instance = pygad.GA(num_generations=10,
                    #    sol_per_pop=5,
                       initial_population=initial_population,
                       num_parents_mating=num_parents_mating,
                       num_genes=(N_STATES + N_STATES**2 + N_STATES*len(ALPHABET)),
                       fitness_func=fitness_func,
                       crossover_type =crossover_func,
                       keep_elitism=1,
                    #    on_stop=on_stop,
                        mutation_func=mutation_func,
                       init_range_high=init_range_high,
                       init_range_low=init_range_low)

ga_instance.run()
ga_instance.plot_fitness()

AttributeError: 'list' object has no attribute 'flatten'